In [ ]:
#### mount BLOB storage
#see https://docs.databricks.com/spark/latest/data-sources/azure/azure-storage.html#mount-azure-blob-storage-containers-with-dbfs

####### CHANGE THE BELOW  4 VALUES   ##########
#destination mount point on the cluster:
blob_mount_location = '/mnt/<mountpoint>'

#container name
container_name = "<some-container>"

# blob storage account name
storage_account_name = "<storage-account-name>"

# get blob key from azure key store
storage_account_access_key = dbutils.secrets.get(scope = "azure-kv-scope", key = "<blob-storage-account-key>")

################################

# configure key
conf_key = 'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net'


# source of blob
blob_source = "wasbs://"+container_name+"@"+storage_account_name+".blob.core.windows.net"

# existing mounts
existing_mounts = dbutils.fs.mounts()


# check if the blob is aleady mounted
checkSource = next((x.source +' at ' + x.mountPoint for x in existing_mounts if blob_source in x.source), 'Not_mounted')

# check if mount point taken
checkMount = next((x.source +' at ' + x.mountPoint for x in existing_mounts if blob_mount_location == x.mountPoint), 'Not_mounted')


if checkSource != 'Not_mounted':
	print("This blob container is already mounted: " + checkSource)

elif checkMount != 'Not_mounted':
	print("The specified mount point is already in use: " + checkMount)
else:
	dbutils.fs.mount(
		source = blob_source,
		mount_point = blob_mount_location,
		extra_configs = {conf_key:storage_account_access_key}
	)
	print(blob_source + ' mounted at ' + blob_mount_location + '!')
	#dbutils.fs.mounts()
  